In [26]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


import keras
from keras import models
from keras import layers
from keras.layers import Activation, Dense

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('data.csv')
data.head()
data.shape
data = data.drop(['filename'], axis = 1)

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1], dtype = float))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

len(y_train)
len(y_test)
X_train[10]

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=20,batch_size=128)

test_lostt, test_acc = model.evaluate(X_test,y_test)

print('test_acc: ', test_acc)





x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

results

predictions = model.predict(X_test)

predictions[0].shape
np.sum(predictions[0])
np.argmax(predictions[0])




Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 2.1877 - acc: 0.2188
Epoch 2/20
800/800 [==============================] - 0s 55us/step - loss: 1.8623 - acc: 0.3287
Epoch 3/20
800/800 [==============================] - 0s 53us/step - loss: 1.6439 - acc: 0.3688
Epoch 4/20
800/800 [==============================] - 0s 51us/step - loss: 1.4861 - acc: 0.4700
Epoch 5/20
800/800 [==============================] - 0s 52us/step - loss: 1.3620 - acc: 0.5375
Epoch 6/20
800/800 [==============================] - 0s 56us/step - loss: 1.2482 - acc: 0.5925
Epoch 7/20
800/800 [==============================] - 0s 52us/step - loss: 1.1481 - acc: 0.6300
Epoch 8/20
800/800 [==============================] - 0s 54us/step - loss: 1.0654 - acc: 0.6687
Epoch 9/20
800/800 [==============================] - 0s 58us/step - loss: 1.0032 - acc: 0.6788
Epoch 10/20
800/800 [==============================] - 0s 54us/step - loss: 0.9438 - acc: 0.6962
Epoch 11/20
800/800 [===================

8